In [ ]:
import h5py
from scipy.io import savemat
from tempfile import TemporaryFile
import numpy as np
import itertools as it
from scipy.integrate import quad
import math as mt
import scipy.special as psi
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
import math as mth
import matplotlib.gridspec as gridspec
blue=(0.00784313725490196, 0.24313725490196078, 1.0)
red=(0.9098039215686274, 0.0, 0.043137254901960784)

In [ ]:
# Integral function for the compuation of entropy to reduce bias
def integrand(t, ni):
     return (t**(ni-1))/(1+t)

In [ ]:
# Computes all the permutations of a range 0-k
def permutation(k):
  f_k=mt.factorial(k)
  A=np.empty((f_k,k))
  for i, perm in enumerate(it.permutations(range(k))):
      A[i,:] = perm
  return A

In [ ]:
# Converts an array into a list
def array_list(array_num): 
    num_list = array_num.tolist()
    return num_list

In [ ]:
# Creates an array that displays which permuation represent the order
# of the input array
  
def ubble(v):
    n=len(v)
    a=range(0,n)
    B=np.array([v,a])
    t=np.array([0., 0.])
    for i in range(0,n-1):
        for j in range(1,n):
            if B[0,j-1]>B[0,j]:
                
                t[:]=B[:,j]
                B[:,j]=B[:,j-1]
                B[:,j-1]=t[:]    
    ord=B[1][:]
    return ord
    

In [ ]:
# Main function, computes the mPE
    
# [H,h]=mPE(data,n)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data"
    
# [H,h]=mPE(data,n,eps)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data". The vector eps contains
# as many elements as the number of scales considered by the algorithm.
# Each element of the vector eps represents how many elements are 
# considered in creating a coarse-grained time series. Default is eps=[1]
    
# [H,h]=mPE(data,n,eps,L)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data". The vector eps contains
# as many elements as the number of scales considered by the algorithm.
# Each element of the vector eps represents how many elements are 
# considered in creating a coarse-grained time series. Default is eps=[1]
# L is a lag parameter, default is 1.
# 
# n(order of the entropy)
# L(distance between two adjacent elements in the computation)(generally=1)
def mPE(*argv):
    
    g=len(argv)
    
    if g==2:
        n=argv[1]
        datain=argv[0]
        L=1
        epsin=np.array([1])
    if g==3:
        n=argv[1]
        datain=argv[0]
        L=1
        epsin=argv[2]
    if g==4:
        n=argv[1]
        datain=argv[0]
        L=argv[3]
        epsin=argv[2]
    
    
    
#    Check for the data to be in the right dimension
    if len(datain[:,1])>len(datain[1,:]):
        datain=datain.transpose()
        
    
    scalesno=len(epsin)
    HH=np.zeros((1,scalesno));
    norm_HH=np.zeros((1,scalesno));
#    Definition of parameters: fac is the number of possible permutations
#    Elem is the No of data points
#    Dim is the dimensionality of the samples
    
    
    for eps in range(0,scalesno):
        scale=epsin[eps]
        xlen=len(datain[1,:])
        ylen=np.round((xlen/scale)-0.5)
        ylen=ylen.astype(int)
        D=np.zeros((len(datain[:,1]),ylen))
        for ylenc in range(0,ylen):
            dfg=datain[:,((ylenc)*scale):((ylenc+1)*scale)]
            fhk=np.sum(dfg,axis=1)
            r=(1/scale)*fhk
            D[:,ylenc]=r
            
        data=D
        fac=mt.factorial(n)
        elem=len(data[1,:])
        dim=len(data[:,1])
        
    #    A is a n-by-factorial(n) matrix that shows all the 
    #    possible permutations of n elements
        A=permutation(n)
        
    #    counter is a factorial(n) square matrix that counts the recurrence of
    #    a dim-dimensional permutation
        lpi=fac*np.ones((dim))
        lpi=lpi.astype(int)
        nj = lpi.tolist()
        counter=np.zeros((nj))
        
    #    For each iteration i, a series of n points is sampled from the 
    #    data set and the corresponding permutation is identified. 
    #    The counter matrix keeps track of the numiber of times a certain
    #    permutation, or combination of permutations is observed. 
        for i in range(0, elem-n*L+L-1):
            coord=np.zeros((dim))
            for num in range(0,dim):
                sample=data[num,range(i,i+L*(n),L)]
                ord=ubble(sample)
                perm_num=0
                check_1=1
                check_2=1
                
                while (perm_num<=fac) and (check_2):
                   
                    check_1=1
                    for j in range(0,n-1):
                        if ord[j]!=A[perm_num,j]:
                            check_1=0
                    if check_1:
                        coord[num]=perm_num
                        check_2=0
                        
                    perm_num=perm_num+1
            
                
            coord=coord.astype(int)
            coord1=tuple(coord)
            counter[coord1]=counter[coord1]+1
            
    #    Once the counter matrix is complete, each element is divided by the
    #    total number of samples to get a empirical probability, and the PE is
    #    computed according to the method described in Schurmann
        
        H=0
        counter1=counter.flatten()
        for iter in range(0,(fac**dim)):
           
            rec=counter1[iter]
            tot=elem-n+1
            
            if rec==0:
                H=H
                
            else:
                I = quad(integrand, 0, 1, args=(rec))
                I=I[0]
                coeff=((-1)**rec)*I
                prob=(rec/tot)*(psi.digamma(tot)-psi.digamma(rec)-coeff)
                H=H+mt.log2(mt.exp(prob))
             
    #     The normalised entropy norm_H is computed as well
        norm_H=H/(mt.log2(fac**dim))
        
        HH[0,eps]=H
        norm_HH[0,eps]=norm_H
    return [HH,norm_HH]

In [ ]:
#Moving average filter function, n is order
def movav(data,n):
    n=n+1
    vo=np.zeros([len(data[:,0])-n+1,len(data[0])]);

    for i in range(0,len(data[:,0])-n+1):
        vec=data[i:i+n-1];
        vo[i]=np.mean(vec, axis=0);
    return(vo)

In [ ]:
##Inputs
order=3
screenwidth=1280
screenheight=960
filename='Age_study_raw.hdf'
groupname='Age study'


f = h5py.File(filename, 'r')
data=f.get(groupname)
f1=h5py.File(filename, 'r')
data1=f1.get(groupname)

In [ ]:
#Definition of variables
sbj=np.array(data['SUBJECTINDEX'])
trial=np.array(data['trial'])
x=np.array(data['x'])
y=np.array(data['y'])
sizem1=np.int(np.amax(sbj))
sizem2=np.int(np.amax(trial))
E=np.zeros([sizem1,sizem2])

In [ ]:
#Define which, and how many, conditions are considered in the trial.
#Label is the name of the correspondent parameter inside "f"
label1='age'
#label2='age'
#label3='age'

par1=np.array(data[label1])
mat1=np.zeros([sizem1,sizem2])
#par2=np.array(data[label2])
#mat2=np.zeros([sizem1,sizem2])
#par3=np.array(data[label3])
#mat3=np.zeros([sizem1,sizem2])

In [ ]:
###### PREPROCESSING
for ko in range(0,len(x[0])):
    
    if x[0,ko]<0 or x[0,ko]>screenwidth:
        if ko==1:
            x[0,ko]=0;
        else:
            x[0,ko]=x[0,ko-1];
            
    if x[0,ko]<0 or x[0,ko]>screenheight:
        if ko==1:
            y[0,ko]=0;
        else:
            y[0,ko]=y[0,ko-1];

In [ ]:
###### ENTROPY AND CONDITIONS
for i in range(np.int(np.amin(sbj,axis=1)),np.int(np.amax(sbj,axis=1)+1)):
    
    trial1=trial[0,sbj[0,:]==i]
    x1=x[0,sbj[0,:]==i]
    y1=y[0,sbj[0,:]==i]
    
    par1_1=par1[0,sbj[0,:]==i]
#    par2_1=par2[0,sbj[0,:]==i]
#    par3_1=par3[0,sbj[0,:]==i]
    for j in range(np.int(np.amin(trial1,axis=0)),np.int(np.amax(trial1,axis=0)+1)):
        
        x2=x1[trial1==j]
        y2=y1[trial1==j]
        d=np.c_[x2,y2]
        par1_2=par1_1[trial1==j]
#        par2_2=par2_1[trial1==j]
#        par3_2=par3_1[trial1==j]
        if d.size==0:
            HH=-1
            mat1[i-1,j-1]=-1
#            mat2[i-1,j-1]=-1
#            mat3[i-1,j-1]=-1
        else:
            [HH,hh]=mPE(d,order)
            mat1[i-1,j-1]=par1_2[0]
#            mat2[i-1,j-1]=par2_2[0]
#            mat3[i-1,j-1]=par3_2[0]
        E[i-1,j-1]=HH

In [ ]:
##OCCUPANCY
oc=np.zeros([sizem1,sizem2])

xpar=np.round(np.arange(0,screenwidth+1,20))
ypar=np.round(np.arange(0,screenheight+1,20))

for i in range(np.int(np.amin(sbj,axis=1)),np.int(np.amax(sbj,axis=1)+1)):
    trial1=trial[0,sbj[0,:]==i]
    x1=x[0,sbj[0,:]==i]
    y1=y[0,sbj[0,:]==i]
    for j in range(np.int(np.amin(trial1,axis=0)),np.int(np.amax(trial1,axis=0)+1)):
        grid=np.zeros([np.int(np.round(screenwidth/20)),np.int(np.round(screenheight/20))])
        x2=x1[trial1==j]
        y2=y1[trial1==j]
        for ii in range(0,len(x2)):
            xpos=x2[ii]
            xv=(xpar[0:len(xpar)-1]<xpos) & (xpar[1:len(xpar)]>=xpos)
            ypos=y2[ii]
            yv=(ypar[0:len(ypar)-1]<ypos) & (ypar[1:len(ypar)]>=ypos)
            grid[xv,yv]=grid[xv,yv]+1
            


            grid[grid>0]=1
            oc[i,j]=sum(sum(grid));

In [ ]:
####EXTRACT PROCESSED DATA
filename='etdb_v1.0.hdf5' #includes al processed data
f1=h5py.File(filename, 'r')
data1=f1.get(groupname)
sbj=np.array(data1['SUBJECTINDEX'])
trial=np.array(data1['trial'])
x=np.array(data1['x'])
y=np.array(data1['y'])
startt=np.array(data1['start'])
endt=np.array(data1['end'])
sizem1=np.int(np.amax(sbj))
sizem2=np.int(np.amax(trial))
ft=np.zeros([sizem1,sizem2])
ss=np.zeros([sizem1,sizem2])

In [ ]:
#SACCADIC SPEED (SS) AND FIXATION TIME (FT)
for i in range(np.int(np.amin(sbj,axis=0)),np.int(np.amax(sbj,axis=0)+1)):
    
    trial1=trial[sbj==i]
    x1=x[sbj==i]
    y1=y[sbj==i]
    startt1=startt[sbj==i]
    endt1=endt[sbj==i]
    for j in range(np.int(np.amin(trial1,axis=0)),np.int(np.amax(trial1,axis=0)+1)):
        
        x2=x1[trial1==j]
        y2=y1[trial1==j]
        startt2=startt1[trial1==j]
        endt2=endt1[trial1==j]
        tim=np.mean(endt2-startt2)
        ft[i-1,j-1]=tim
        
        endt3=endt2[0:len(endt2)-1]
        startt3=startt2[1:len(startt2)]
        dur=-endt3+startt3
        distx=abs(x2[1:len(x2)]-x2[0:len(x2)-1])
        disty=abs(y2[1:len(y2)]-y2[0:len(y2)-1])
        dist=(disty**2+distx**2)**0.5
        speed=dist/dur
        speed=np.mean(speed)
        ss[i-1,j-1]=speed

In [ ]:
##SAVE FT, ENTROPY, OCCUPANCY,SS IN CURRENT WORKSPACE
savemat(groupname+'_results.mat', mdict={'entr':E,'par1':mat1,'occupancy':oc,'saccadic_speed':ss, 'fixation_time':ft})

In [ ]:
##EXAMPLE OF DATA VISUALISATION
groupname='Age study'
f = loadmat(groupname+'_results.mat')
entr=f['entr']
age=f['par1']
old=entr[age==2]
young=entr[age==0]

fig, axes = plt.subplots(2,2, sharex=False,sharey=False)
fig.set_size_inches(20,10)
sns.distplot(old,ax=axes[0,0], bins=40, kde=True, hist = tv, label=None, norm_hist=True, kde_kws={'color': blue,"alpha":1, "lw": 3, "label": "Old"})
sns.distplot(young,ax=axes[0,0], bins=40, kde=True, hist = tv, label=None, norm_hist=True, kde_kws={'color': red,"alpha":1, "lw": 3, "label": "Young"})
